In [7]:
from time import time
from datetime import datetime
import numpy as np
seq = 13
dt = 8  
def multiply(A,B):
    return [a*b for a,b in zip(A,B)]
def divide(A,B):
    return [a/b for a,b in zip(A,B)]

def minus(A,B):
    return [a-b for a,b in zip(A,B)]
def log(txt):
    f= open("log%i.txt"%seq,"a+")
    f.write('%s: %s \r\n'%(datetime.now().strftime('{%Y-%m-%d %H:%M:%S}'),txt))
    print(txt)
    f.close() 
def initial_log():
    f= open("log%i.txt"%seq,"w+")
    f.write('%s: starting.... \r\n'%(datetime.now().strftime('{%Y-%m-%d %H:%M:%S}')))
    f.close() 
import pickle

def savedf():
    with open('df%i.dat'%dt,'wb') as f:
        pickle.dump((X_train, X_test, y_train, y_test),f)

def loaddf():
    global X_train, X_test, y_train, y_test
    with open('df%i.dat'%dt,'rb') as f:
        X_train, X_test, y_train, y_test = pickle.load(f)
    print("Shape of X_train:", len(X_train),len(X_train[0]))
    print("Shape of X_test:",len(X_test),len(X_test[0]))

      
def save():
    with open('data%i.dat'%seq,'wb') as f:
        pickle.dump((i,mdl_1,mdl_2,mdl_3,RMSETest,RMSETrain,last_depth,last_rmse),f)
def load():
    global i,mdl_1,mdl_2,mdl_3,RMSETest,RMSETrain,last_depth,last_rmse
    with open('data%i.dat'%seq,'rb') as f:
        i,mdl_1,mdl_2,mdl_3,RMSETest,RMSETrain,last_depth,last_rmse = pickle.load(f)

def load1():
    global i,mdl_1,mdl_2,mdl_3,RMSETest,RMSETrain,last_depth,last_rmse
    with open('data4.dat','rb') as f:
        i,mdl_1,mdl_2,mdl_3,RMSETest,RMSETrain,last_depth,last_rmse = pickle.load(f)
import os
os.environ["PATH"] += os.pathsep +"C:\\Users\\User\\Downloads\\graphviz-2.38\\release\\bin"


In [8]:
from numpy import var

class Splitter:
    def __init__(self,X,Y):
        self.X = X
        self.Y = Y

    def split(self,index,value):
        """ 
        Split X and Y based on the given variable index and value
  
        Parameters: 
            index (int): index of variable to splitted on
            value (float): the value of splitting varaible
          
        Returns: 
            right_x,left_x,rigth_y,left_y
        """
        right_x = [x for x in self.X if x[index]>=value]
        left_x = [x for x in self.X if x[index]<value]
        
        right_y = [self.Y[i] for i,x in enumerate(self.X) if x[index]>=value]
        left_y = [self.Y[i] for i,x in enumerate(self.X) if x[index]<value]
        return right_x,left_x,right_y,left_y
    
    def weighted_var(self,Y):
        if len(Y) ==0:
            return float('inf')
        return var(Y)/len(Y)
    
    def SE(self,Y):
        if len(Y) ==0: return float('inf')
        mean = sum(Y)/len(Y)
        SE = sum([(y-mean)**2 for y in Y])
        return SE
        
    
    def splitting_cost(self,right_x,left_x,right_y,left_y):
        """ 
        Compute cost of spliting.
        in the current version, we have used varaince based cost function
  
        Parameters: 
            right_x,left_x,rigth_y,left_y
          
        Returns: 
            float: the cost of computing
        """        
        return (self.SE(right_y)+self.SE(left_y))/(len(right_y)+len(left_y))
    
    def possibleValues(self,index):
        values = [x[index] for x in self.X]
        values.sort()
        max_values = 100
        if len(values)<max_values:
            return values
        
        return set([np.percentile(values,i*100//max_values) for i in range(max_values)])
    
    def test_splitting(self,index,value):
        right_x,left_x,right_y,left_y = self.split(index,value)
        return  self.splitting_cost(right_x,left_x,right_y,left_y),len(right_y),len(left_y)
    
    def find_best_splitting(self,min_leaf_treshold=0,min_split=0):
        """ 
        Find best splitting and return index and value of splitting variable respectively
       
        Returns: 
            int,float: the index and value of best possible splitting variables
        """ 
        
        best_splitting = (self.SE(self.Y)/len(self.Y))
        best_splitting_index = -1
        best_splitting_value = 0
        if len(self.Y)<min_split:
            return best_splitting_index,best_splitting_value            
        for index in range(len(self.X[0])):
            for value in self.possibleValues(index):
                splitting_cost,left_cnt,right_cnt = self.test_splitting(index,value)
#               print('splitting at %f: cost->%f'%(value,splitting_cost))
                if splitting_cost < best_splitting and left_cnt >= min_leaf_treshold and right_cnt >= min_leaf_treshold:
                    best_splitting = splitting_cost
                    best_splitting_index = index
                    best_splitting_value = value
        return best_splitting_index,best_splitting_value


In [9]:
id = 0
last_leaf_no = 1
j = 0
def create_Tree():
    global last_leaf_no
    last_leaf_no = 1
    return Node(0)
class Node:
    isLeaf = True
    Left = None
    Right = None
    Value = None
    Depth = 0
    splitting_var_index = 0
    splitting_value = 0
    cnt = 0
    
    
    def __init__(self,depth):
        global id
        id+=1
        self.ID = id
        self.isLeaf = True
        self.Depth = depth
    def predict(self,X):
        return [self.predict_one(x).Value for x in X]
        
    def predict_one(self,X):
        if type(X) != list:
            raise TypeError("predict_one.X: expected list but recieved %s (%s)"%(X,type(X)))
        try:
            if self.isLeaf:
                self.cnt +=1
                return self
            if X[self.splitting_var_index] >= self.splitting_value:
                return self.Right.predict_one(X)
            else:
                return self.Left.predict_one(X)
        except IndexError:
            print('index: %i'%self.splitting_var_index)
            raise
    def get_leafs(self):
        if self.isLeaf:
            return [self]
        return self.Right.get_leafs()+self.Left.get_leafs()   
    
    def fit(self,X,Y,max_depth):
        global last_leaf_no

        #tree reached maximum depth
        if max_depth <= self.Depth:
            self.isLeaf = True
            self.leafID = last_leaf_no
            last_leaf_no+=1
            self.fit_leaf_value(Y)
            return 
        
        #continue splitting if more depth allowed
        splitter = Splitter(X,Y)
        best_splitting_index,best_splitting_value = splitter.find_best_splitting()
        if best_splitting_index == -1:
            #better to not split
            self.isLeaf = True
            self.leafID = last_leaf_no
            last_leaf_no+=1
            self.fit_leaf_value(Y)
        else:
            right_x,left_x,right_y,left_y = splitter.split(best_splitting_index,best_splitting_value)
            self.isLeaf = False
            self.splitting_var_index = best_splitting_index
            self.splitting_value = best_splitting_value
            self.Right = Node(self.Depth+1)
            self.Left = Node(self.Depth+1)
            self.Right.fit(right_x,right_y,max_depth)
            self.Left.fit(left_x,left_y,max_depth)
    def fit_leaf_value(self,Y):
        self.Value = sum(Y)/len(Y)

        
    def tune(self,X,R,T,bound=(-0.01,.1)):
        #self.X = X
       # self.Y = ['%f:%f'%(r,t) for r,t in zip(R,T)]
        if len(X) != len(R):
            raise TypeError("tune function: X and R should be with same size. recieved <%i>.<%i>"%(len(X),len(R)))
        if len(X) != len(T):
            raise TypeError("tune function: X and T should be with same size. recieved <%i>.<%i>"%(len(X),len(T)))
            
        if self.isLeaf:
            if sum(multiply(T,T)) == 0:
                self.value = 1
            else:
                self.Value = sum(multiply(R,T))/sum(multiply(T,T))
                self.Value = min(bound[1],max(bound[0],self.Value))
            return
        
        splitter = Splitter(X,R)
        right_x,left_X,right_R,left_R = splitter.split(self.splitting_var_index,self.splitting_value)
        splitter = Splitter(X,T)
        right_x,left_X,right_T,left_T = splitter.split(self.splitting_var_index,self.splitting_value)
        self.Right.tune(right_x,right_R,right_T)
        self.Left.tune(left_X,left_R,left_T)
        
    def draw_tree(self):
        global j
        from graphviz import Source
        from io import BytesIO
        import time
        timestr = time.strftime("%Y%m%d-%H%M%S")
        from matplotlib import pyplot as plt
        from matplotlib import image
        g = Source('digraph {\n\tgraph [rankdir=UT]\n'+self.print_tree()+'}')
        g.render('round-table%i.gv'%j, view=True)
        j+=1
        #return img
        
    def get_leafcounts(self):
        if self.isLeaf:
            return 1
        return self.Left.get_leafcounts()+self.Right.get_leafcounts()
    
    def print_tree(self,prefix=''):
        '\t0 [label="f0<8.40779079e-45] yes=1,no=2,missing=1\n\t1:[f0<2.80259693e-45] yes=3,no=4,missing=3\n\t\t3:leaf=7.46349096\n\t\t4:leaf=-0.278686523\n\t2:[f0<8.40779079e-45] yes=5,no=6,missing=5\n\t\t5:leaf=-0\n\t\t6:leaf=-0.188493282\n'
        if not self.isLeaf:
            ret = '\t%i [label="f%i>=%f"]\n\t%i -> %i [label="yes" color="#0000FF"]\n\t %i -> %i [label=no color="#FF0000"]\n'%(
                self.ID,self.splitting_var_index,self.splitting_value,self.ID,self.Right.ID,self.ID,self.Left.ID)
            ret += self.Right.print_tree(prefix+'\t')
            ret += self.Left.print_tree(prefix+'\t')
            return ret
        else:
            return '\t%i [label="leaf=%f\nCount=%i"]\n'%(self.ID,self.Value,self.cnt)
    def get_leaf_values(self):
        if self.isLeaf:
            return [self.Value]
        return self.Right.get_leaf_values()+self.Left.get_leaf_values()
    
'''digraph {
	graph [rankdir=UT]
	0 [label="f0<8.40779079e-45"]
	0 -> 1 [label="yes, missing" color="#0000FF"]
	0 -> 2 [label=no color="#FF0000"]
	1 [label="f0<2.80259693e-45"]
	1 -> 3 [label="yes, missing" color="#0000FF"]
	1 -> 4 [label=no color="#FF0000"]
	3 [label="leaf=7.46349096"]
	4 [label="leaf=-0.278686523"]
	2 [label="f0<8.40779079e-45"]
	2 -> 5 [label="yes, missing" color="#0000FF"]
	2 -> 6 [label=no color="#FF0000"]
	5 [label="leaf=-0"]
	6 [label="leaf=-0.188493282"]
}'''       
        

'digraph {\n\tgraph [rankdir=UT]\n\t0 [label="f0<8.40779079e-45"]\n\t0 -> 1 [label="yes, missing" color="#0000FF"]\n\t0 -> 2 [label=no color="#FF0000"]\n\t1 [label="f0<2.80259693e-45"]\n\t1 -> 3 [label="yes, missing" color="#0000FF"]\n\t1 -> 4 [label=no color="#FF0000"]\n\t3 [label="leaf=7.46349096"]\n\t4 [label="leaf=-0.278686523"]\n\t2 [label="f0<8.40779079e-45"]\n\t2 -> 5 [label="yes, missing" color="#0000FF"]\n\t2 -> 6 [label=no color="#FF0000"]\n\t5 [label="leaf=-0"]\n\t6 [label="leaf=-0.188493282"]\n}'

In [25]:
a = [0,1,2,3,4,5,6,7,8]
a[:len(a)-2]+a[9-1:]

[0, 1, 2, 3, 4, 5, 6, 8]

In [34]:
    from copy import deepcopy
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    from scipy.sparse import csr_matrix
    from scipy import sparse
    import numpy as np
    from scipy.optimize import lsq_linear
    from sklearn.model_selection import train_test_split

    class FractBoosting:

        def __init__(self,learning_rate):
            self.learning_rate = learning_rate
            self.trees = []

        def predict(self,X,ignore=-1):
            Y = []
            tree_filtered = self.trees
            if ignore >=0:
                tree_filtered = self.trees[:ignore]+self.trees[ignore+1:]
            print('prediciting using %i trees'%len(tree_filtered))
            for x in X:
                predictions = [S.predict([x])[0]*T.predict([x])[0] for S,T in tree_filtered]
                predict = predictions[0] + self.learning_rate*sum(predictions[1:])
                Y.append(predict)

            return Y

        def create_root_tree(self,Y):
            root_s = create_Tree()
            root_s.Value = 1
            root_t = create_Tree()
            root_t.fit_leaf_value(Y)
            self.trees = [(root_s,root_t)]

        def fit_one_step(self,X_TRAIN,Y_TRAIN,depth,backprop_level):
            X,Y = X_TRAIN, Y_TRAIN
            Y_r = self.get_residual(X,Y)
            T = create_Tree()
            T.fit(X,Y_r,depth)
            Y_t = T.predict(X)
            S = deepcopy(self.trees[-1][1])
            S.tune(X,Y_r,Y_t)
            self.trees.append((S,T))
            self.back_prop(X,Y,backprop_level)
            
            
        def back_prop(self,X,Y,level):
            if (len(self.trees)-1) < level:
                return
            print('RMSE before: ',self.rmse(X,Y))
            Y_t = self.trees[-level][1].predict(X)
            h = self.predict(X,len(self.trees)-level)
            Y_r = np.array(minus(Y , h))
            S = deepcopy(self.trees[-level+1][1])
            S.tune(X,Y_r,Y_t)
            self.trees[-level] = (S,self.trees[-level][1])
            print('RMSE before: ',self.rmse(X,Y))

            
        def get_residual(self,X,Y):
            h = self.predict(X)
            Y_r = minus(Y , h)
            return Y_r
        def print_trees(self):
            for i,(s,t) in enumerate(self.trees):
                print('%ith fractal tree\n'%i)
                print('-----------------\n')
                print('S:\n')
                s.print_tree()
                print('-----------------\n')
                print('T:\n')
                t.print_tree()
                print('\n-----------------\n')

        def rmse(self,X,Y):
            h = self.predict(X)
    #        R = minus(Y , h)
    #        print('Residual:',R)
            return sqrt(mean_squared_error(h, Y))


In [ ]:
from numpy import std
import xgboost as xgb
from threading import Thread
from xgboost import plot_tree
from sklearn.metrics import mean_squared_error
max_depth=5

loaddf()
#initial_log()
mdl_1 = FractBoosting(1)
mdl_2 = FractBoosting(1)
mdl_3 = FractBoosting(1)
mdl_1.create_root_tree(y_train)
mdl_2.create_root_tree(y_train)
mdl_3.create_root_tree(y_train)
last_rmse = mdl_3.rmse(X_train,y_train)
last_depth = 1

m = sum(y_train)/len(y_train)
y_train_xgb = [y-m for y in y_train]

RMSETest = [[],[],[],[],[],[]]
RMSETrain =  [[],[],[],[],[],[]]
i=0
try:
    load()
    i
except Exception as e:
    print(e)
    initial_log()
try:
    while i<200:
        processes = []




        ################Fixed depth
        mdl_1.fit_one_step(X_train,y_train,max_depth,1)
        RMSETrain[0].append( mdl_1.rmse(X_train,y_train))
        RMSETest[0].append( mdl_1.rmse(X_test,y_test))    
        print('1 Back')


        
        ###############Linear depth
        #depth = min(5,int(1+max_depth*(i/50)))
        mdl_2.fit_one_step(X_train,y_train,max_depth,2)  
        RMSETrain[1].append( mdl_2.rmse(X_train,y_train))
        RMSETest[1].append( mdl_2.rmse(X_test,y_test))    
        print('2 Back')


        ###############dynamic depth
        mdl_3.fit_one_step(X_train,y_train,max_depth,3)
        #mdl3_rmse = mdl_3.rmse(X_test,y_test)
        RMSETrain[2].append( mdl_3.rmse(X_train,y_train))
        RMSETest[2].append( mdl_3.rmse(X_test,y_test))
        print('3 Back')
        


        ##################XGBoost
        xg_reg = xgb.XGBRegressor( learning_rate = 0.1,
                        max_depth = 5,  n_estimators = i)
        xg_reg.fit(X_train,y_train_xgb)    
        preds = xg_reg.predict(X_train)
        RMSETrain[3].append(np.sqrt(mean_squared_error(y_train, [p+m for p in preds])))
        preds = xg_reg.predict(X_test)
        RMSETest[3].append(np.sqrt(mean_squared_error(y_test, [p+m for p in preds])))
        print('XGB')
        i+=1
        
        save()
                           
        log('\r\n-----------------------------------------------\r\n'+
            'ittr %i  \tTrain\t\tTest\t\tDiff\t\tRVar\tSVar\tTVar\r\n'%i+
            'FGB[%i]:  \t%.3f\t\t%.3f\t\t%.3f\t\t%.3f\t%.3f\t\t%.3f\r\n'%(1,RMSETrain[0][-1],RMSETest[0][-1],RMSETest[0][-1]-RMSETrain[0][-1],std(mdl_1.get_residual(X_train,y_train)),std(mdl_1.trees[-1][0].get_leaf_values()),std(mdl_1.trees[-1][1].get_leaf_values()))+
            'FGB[%i]:  \t%.3f\t\t%.3f\t\t%.3f\t\t%.3f\t%.3f\t\t%.3f\r\n'%(2,RMSETrain[1][-1],RMSETest[1][-1],RMSETest[1][-1]-RMSETrain[1][-1],std(mdl_2.get_residual(X_train,y_train)),std(mdl_2.trees[-1][0].get_leaf_values()),std(mdl_2.trees[-1][1].get_leaf_values()))+
            'FGB[%i]:  \t%.3f\t\t%.3f\t\t%.3f\t\t%.3f\t%.3f\t\t%.3f\r\n'%(3,RMSETrain[2][-1],RMSETest[2][-1],RMSETest[2][-1]-RMSETrain[2][-1],std(mdl_3.get_residual(X_train,y_train)),std(mdl_3.trees[-1][0].get_leaf_values()),std(mdl_3.trees[-1][1].get_leaf_values()))+
            'XGB[%i]:  \t%.3f\t\t%.3f\t\t%.3f \r\n'%(max_depth,RMSETrain[3][-1],RMSETest[3][-1],RMSETest[3][-1]-RMSETrain[3][-1]))


except Exception as e:
    log(e)
    raise


Shape of X_train: 22865 9
Shape of X_test: 22865 9
prediciting using 1 trees
[Errno 2] No such file or directory: 'data13.dat'
prediciting using 1 trees
prediciting using 2 trees
RMSE before:  5.940252236303511
prediciting using 1 trees
prediciting using 2 trees
RMSE before:  5.940252236303511
prediciting using 2 trees
prediciting using 2 trees
1 Back
prediciting using 1 trees
prediciting using 2 trees
prediciting using 2 trees
2 Back
prediciting using 1 trees
prediciting using 2 trees
prediciting using 2 trees
3 Back
[12:31:27] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGB
prediciting using 2 trees
prediciting using 2 trees
prediciting using 2 trees

-----------------------------------------------
ittr 1  	Train		Test		Diff		RVar	SVar	TVar
FGB[1]:  	5.940		5.950		0.009		5.940	0.000		3.955
FGB[2]:  	5.940		5.950		0.009		5.940	0.000		3.955
FGB[3]:  	5.940		5.950		0.009		5.940	0

prediciting using 9 trees
RMSE before:  5.200871125273352
prediciting using 9 trees
prediciting using 9 trees
1 Back
prediciting using 8 trees
prediciting using 9 trees
RMSE before:  5.186905573635687
prediciting using 8 trees
prediciting using 9 trees
RMSE before:  5.1868972588421505
prediciting using 9 trees
prediciting using 9 trees
2 Back
prediciting using 8 trees
prediciting using 9 trees
RMSE before:  5.190613542453549
prediciting using 8 trees
prediciting using 9 trees
RMSE before:  5.190308340228864
prediciting using 9 trees
prediciting using 9 trees
3 Back
[13:06:25] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGB
prediciting using 9 trees
prediciting using 9 trees
prediciting using 9 trees

-----------------------------------------------
ittr 8  	Train		Test		Diff		RVar	SVar	TVar
FGB[1]:  	5.201		5.263		0.062		5.201	0.000		2.968
FGB[2]:  	5.187		5.251		0.064		5.187	0.0

prediciting using 16 trees
RMSE before:  4.834327687251406
prediciting using 15 trees
prediciting using 16 trees
RMSE before:  4.834394956358579
prediciting using 16 trees
prediciting using 16 trees
1 Back
prediciting using 15 trees
prediciting using 16 trees
RMSE before:  4.8303391632533295
prediciting using 15 trees
prediciting using 16 trees
RMSE before:  4.830843026162261
prediciting using 16 trees
prediciting using 16 trees
2 Back
prediciting using 15 trees
prediciting using 16 trees
RMSE before:  4.823413896834436
prediciting using 15 trees
prediciting using 16 trees
RMSE before:  4.823950878470171
prediciting using 16 trees
prediciting using 16 trees
3 Back
[13:50:24] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGB
prediciting using 16 trees
prediciting using 16 trees
prediciting using 16 trees

-----------------------------------------------
ittr 15  	Train		Test		Diff		

prediciting using 22 trees
prediciting using 22 trees

-----------------------------------------------
ittr 21  	Train		Test		Diff		RVar	SVar	TVar
FGB[1]:  	4.643		4.785		0.143		4.643	0.000		2.132
FGB[2]:  	4.653		4.804		0.150		4.653	0.041		2.786
FGB[3]:  	4.651		4.799		0.148		4.651	0.049		3.671
XGB[5]:  	4.666		4.817		0.151 

prediciting using 22 trees
prediciting using 23 trees
RMSE before:  4.620479747719787
prediciting using 22 trees
prediciting using 23 trees
RMSE before:  4.62069783788965
prediciting using 23 trees
prediciting using 23 trees
1 Back
prediciting using 22 trees
prediciting using 23 trees
RMSE before:  4.632484612309109
prediciting using 22 trees
prediciting using 23 trees
RMSE before:  4.632541752112434
prediciting using 23 trees
prediciting using 23 trees
2 Back
prediciting using 22 trees
prediciting using 23 trees
RMSE before:  4.621708736808613
prediciting using 22 trees
prediciting using 23 trees
RMSE before:  4.621629980482592
prediciting using 23 trees
predici

prediciting using 29 trees
RMSE before:  4.484349312509206
prediciting using 29 trees
prediciting using 29 trees
3 Back
[15:17:34] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGB
prediciting using 29 trees
prediciting using 29 trees
prediciting using 29 trees

-----------------------------------------------
ittr 28  	Train		Test		Diff		RVar	SVar	TVar
FGB[1]:  	4.497		4.673		0.176		4.497	0.000		3.352
FGB[2]:  	4.510		4.692		0.182		4.510	0.053		2.662
FGB[3]:  	4.484		4.669		0.185		4.484	0.032		3.109
XGB[5]:  	4.497		4.687		0.191 

prediciting using 29 trees
prediciting using 30 trees
RMSE before:  4.486504241834007
prediciting using 29 trees
prediciting using 30 trees
RMSE before:  4.487024396930316
prediciting using 30 trees
prediciting using 30 trees
1 Back
prediciting using 29 trees
prediciting using 30 trees
RMSE before:  4.49624864427074
prediciting using 29 trees
prediciting

...

XGB
prediciting using 163 trees
prediciting using 163 trees
prediciting using 163 trees

-----------------------------------------------
ittr 162  	Train		Test		Diff		RVar	SVar	TVar
FGB[1]:  	3.591		4.130		0.538		3.591	0.000		2.504
FGB[2]:  	3.604		4.143		0.539		3.604	0.053		3.595
FGB[3]:  	3.597		4.137		0.540		3.597	0.048		3.074
XGB[5]:  	3.709		4.217		0.507 

prediciting using 163 trees
prediciting using 164 trees
RMSE before:  3.5877930610104025
prediciting using 163 trees
prediciting using 164 trees
RMSE before:  3.587815563204631
prediciting using 164 trees
prediciting using 164 trees
1 Back
prediciting using 163 trees
prediciting using 164 trees
RMSE before:  3.6021531446609014
prediciting using 163 trees
prediciting using 164 trees
RMSE before:  3.602188080252997
prediciting using 164 trees
prediciting using 164 trees
2 Back
prediciting using 163 trees
prediciting using 164 trees
RMSE before:  3.5914317666162416
prediciting using 163 trees
prediciting using 164 trees
RMSE before

prediciting using 170 trees
RMSE before:  3.5769552153495128
prediciting using 170 trees
prediciting using 170 trees
2 Back
prediciting using 169 trees
prediciting using 170 trees
RMSE before:  3.566390360145066
prediciting using 169 trees
prediciting using 170 trees
RMSE before:  3.566383412201862
prediciting using 170 trees
prediciting using 170 trees
3 Back
[16:47:29] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGB
prediciting using 170 trees
prediciting using 170 trees
prediciting using 170 trees

-----------------------------------------------
ittr 169  	Train		Test		Diff		RVar	SVar	TVar
FGB[1]:  	3.564		4.118		0.553		3.564	0.000		2.511
FGB[2]:  	3.577		4.133		0.556		3.577	0.055		3.398
FGB[3]:  	3.566		4.122		0.556		3.566	0.055		3.445
XGB[5]:  	3.693		4.210		0.518 

prediciting using 170 trees
prediciting using 171 trees
RMSE before:  3.559220536879597
prediciting using 170

prediciting using 177 trees
RMSE before:  3.536604513374349
prediciting using 176 trees
prediciting using 177 trees
RMSE before:  3.536690572669014
prediciting using 177 trees
prediciting using 177 trees
1 Back
prediciting using 176 trees
prediciting using 177 trees
RMSE before:  3.556289067353631
prediciting using 176 trees
prediciting using 177 trees
RMSE before:  3.556286509270234
prediciting using 177 trees
prediciting using 177 trees
2 Back
prediciting using 176 trees
prediciting using 177 trees
RMSE before:  3.5403455726753874
prediciting using 176 trees
prediciting using 177 trees
RMSE before:  3.5402866780614537
prediciting using 177 trees
prediciting using 177 trees
3 Back
[18:27:15] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGB
prediciting using 177 trees
prediciting using 177 trees
prediciting using 177 trees

-----------------------------------------------
ittr 176

XGB
prediciting using 183 trees
prediciting using 183 trees
prediciting using 183 trees

-----------------------------------------------
ittr 182  	Train		Test		Diff		RVar	SVar	TVar
FGB[1]:  	3.516		4.097		0.581		3.516	0.000		2.311
FGB[2]:  	3.536		4.119		0.583		3.536	0.055		3.179
FGB[3]:  	3.527		4.110		0.583		3.527	0.052		2.225
XGB[5]:  	3.650		4.197		0.547 

prediciting using 183 trees
prediciting using 184 trees
RMSE before:  3.513953868470823
prediciting using 183 trees
prediciting using 184 trees
RMSE before:  3.513975957174152
prediciting using 184 trees
prediciting using 184 trees
1 Back
prediciting using 183 trees
prediciting using 184 trees
RMSE before:  3.5330862434503256
prediciting using 183 trees
prediciting using 184 trees
RMSE before:  3.533078629740466
prediciting using 184 trees
prediciting using 184 trees
2 Back
prediciting using 183 trees
prediciting using 184 trees
RMSE before:  3.5241660552504266
prediciting using 183 trees
prediciting using 184 trees
RMSE before:

prediciting using 190 trees
RMSE before:  3.5178636503607335
prediciting using 190 trees
prediciting using 190 trees
2 Back
prediciting using 189 trees
prediciting using 190 trees
RMSE before:  3.508002924698901
prediciting using 189 trees
prediciting using 190 trees
RMSE before:  3.5080306432402044
prediciting using 190 trees
prediciting using 190 trees
3 Back
[21:39:31] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGB
prediciting using 190 trees
prediciting using 190 trees
prediciting using 190 trees

-----------------------------------------------
ittr 189  	Train		Test		Diff		RVar	SVar	TVar
FGB[1]:  	3.494		4.090		0.596		3.494	0.000		1.757
FGB[2]:  	3.518		4.113		0.595		3.518	0.054		2.109
FGB[3]:  	3.508		4.104		0.596		3.508	0.053		2.648
XGB[5]:  	3.626		4.188		0.563 

prediciting using 190 trees
prediciting using 191 trees
RMSE before:  3.4913744670882667
prediciting using 1

In [319]:
print(X)
print(minus(Y,mdl.predict(X,1)))
for x,y in zip(X,minus(Y,mdl.predict(X,1))):
    if x[0]<8:
        print(x,y)

[[1], [7], [8], [3], [4], [10], [6], [8], [5], [3]]
prediciting using 2 trees
[0.0, 0.0, 3.333333333333333, 1.0, 0.0, 1.333333333333333, 1.0, -4.666666666666667, 0.0, -2.0]
prediciting using 2 trees
[1] 0.0
[7] 0.0
[3] 1.0
[4] 0.0
[6] 1.0
[5] 0.0
[3] -2.0


In [193]:
import urllib# url of the data
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00265/CASP.csv'
# retrive data from the url and store in the folder name Data with filename as CASP.csv
urllib.request.urlretrieve (url, filename="CASP.csv" )

('CASP.csv', <http.client.HTTPMessage at 0x1b6360ea088>)

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split 
# Stratified sampling
from sklearn.model_selection import StratifiedShuffleSplit
df = pd.read_csv("CASP.csv")
labels = [1,2,3,4,5]
df['RMSD_cat'] = pd.cut(df['RMSD'], 5, labels=labels)
df.head()
# Create the instance from object
split = StratifiedShuffleSplit(n_splits=1, test_size=0.50, random_state=42)

# Create Train - Test Set
for train_index, test_index in split.split(df, df["RMSD_cat"]):
    train_set = df.loc[train_index]
    test_set = df.loc[test_index]

# Removing the RMSD_cat to give the original data
for _set in (train_set, test_set):
    _set.drop(["RMSD_cat"], axis=1, inplace=True)
    
# Creating new copies of Stratified Train and Test data
y_train = train_set["RMSD"].copy().values.tolist()
X_train = train_set.drop("RMSD", axis=1).values.tolist()

y_test = test_set["RMSD"].copy().values.tolist()
X_test = test_set.drop("RMSD", axis=1).values.tolist()
print("Shape of X_train:", len(X_train),len(X_train[0]))
print("Shape of X_test:",len(X_test),len(X_test[0]))
savedf()
type(X_train)

Shape of X_train: 22865 9
Shape of X_test: 22865 9


list

In [ ]:
def removeXY(node):
    if node == None:
        return
    node.X = node.Y = ''
    removeXY(node.Left)
    removeXY(node.Right)
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(500, 500))
removeXY(mdl.trees[85][0])
mdl.trees[85][0].draw_tree(ax=ax)
plt.show()

